In [3]:
using Pkg
Pkg.activate("..")

# Load module
push!(LOAD_PATH, "../src")
using WriterVerifier

using Random
Random.seed!(42);

  Activating project at `c:\Users\igorp\WriterVerifier`


In [8]:
# Load data
println("Loading images...")

image_folder = "../data/words"

# Load images
writers = load_images(image_folder; max_per_writer=100);

Loading images...
Loaded 76 writers
f01: 100 images
d01: 100 images
g05: 100 images
e07: 100 images
e02: 100 images
a04: 100 images
p02: 100 images
p06: 100 images
r02: 100 images
h07: 100 images
b04: 100 images
k02: 100 images
d03: 100 images
d06: 100 images
c03: 100 images
b01: 100 images
c04: 100 images
f03: 100 images
c02: 100 images
g06: 100 images
l07: 100 images
j07: 100 images
k04: 100 images
e06: 100 images
l03: 100 images
a03: 100 images
r03: 100 images
n06: 100 images
j04: 100 images
r06: 100 images
n02: 100 images
g03: 100 images
k01: 86 images
j06: 100 images
h04: 100 images
m02: 100 images
b05: 100 images
g02: 100 images
k03: 100 images
c01: 100 images
f02: 100 images
p03: 100 images
p01: 100 images
a05: 100 images
d05: 100 images
m01: 100 images
g04: 100 images
h06: 100 images
f04: 100 images
n01: 100 images
j01: 100 images
b02: 100 images
b03: 100 images
l04: 100 images
b06: 100 images
n04: 100 images
e01: 100 images
l01: 100 images
a02: 100 images
a01: 100 images
h02: 

In [ ]:
# Create pairs
pairs, labels = create_pairs(writers; positive=100, negative=100);

Creating positive pairs...
Creating negative pairs...
Created 99 pairs
Positive: 49
Negative: 50


In [10]:
# Create model
model = create_model()
println("Model created")


Model created


In [ ]:
# Model training

trained_model, history = train_model!(
    model, pairs, labels;
    epochs=5, 
    batch_size=8,
    learning_rate=0.00001
)

final_acc = round(history["val_acc"][end] * 100, digits=1)
println("Final accuracy: $(final_acc)%")

Starting training...
Training data: 79
Validation data: 20

Epoch 1/5
Train loss: 0.6923
Val loss: 0.6924
Val accuracy: 55.0%

Epoch 2/5
Train loss: 0.6916
Val loss: 0.6926
Val accuracy: 55.0%

Epoch 3/5
Train loss: 0.6864
Val loss: 0.6927
Val accuracy: 45.0%

Epoch 4/5
Train loss: 0.696
Val loss: 0.6928
Val accuracy: 40.0%

Epoch 5/5
Train loss: 0.7022
Val loss: 0.6924
Val accuracy: 50.0%

Training completed!
Final accuracy: 50.0%


In [ ]:
# Testing on real examples

# Select random test examples
idx = randperm(length(pairs))[1:min(6, length(pairs))]
test_pairs = pairs[idx]
test_labels = labels[idx]

for i in 1:length(test_pairs)
    path1, path2 = test_pairs[i]
    true_label = test_labels[i]
    
    # Extract writer names
    writer1 = split(basename(path1), "-")[1]
    writer2 = split(basename(path2), "-")[1]
    
    # Test similarity
    similarity = test_similarity(trained_model, path1, path2)
    
    # Interpret results
    prediction = similarity > 0.5 ? "SAME" : "DIFFERENT"
    truth = true_label == 1 ? "SAME" : "DIFFERENT"
    correct = (similarity > 0.5) == (true_label == 1) ? "✓" : "X"
    
    println("$i. $writer1 and $writer2")
    println("   Similarity: $(round(similarity, digits=3))")
    println("   Prediction: $prediction | Truth: $truth $correct")
end


In [ ]:
# Model saving
try
    if !isdir("../models")
        mkdir("../models")
    end
    
    save_model(trained_model, "../models/model.jld2")
    
catch e
    println("Save error: $e")
end

In [ ]:
# Load my data
println("Loading images...")

image_folder = "../data/my_words"

# Load images
writers = load_images(image_folder; max_per_writer=10000);

In [ ]:
model = load_model("../models/model.jld2")

In [ ]:
pairs, labels = create_pairs(writers, positive=4, negative=4)

In [ ]:
for i in 1:length(pairs)
    true_label = labels[i]
    path1, path2 = pairs[i]

    # Extract writer names
    writer1 = split(basename(path1), "-")[1]
    writer2 = split(basename(path2), "-")[1]

    similarity = test_similarity(model, path1, path2)


    # Interpret results
    prediction = similarity > 0.5 ? "SAME" : "DIFFERENT"
    truth = true_label == 1 ? "SAME" : "DIFFERENT"
    correct = (similarity > 0.5) == (true_label == 1) ? "✓" : "X"

    println("$i. $writer1 and $writer2")
    println("   Similarity: $(round(similarity, digits=3))")
    println("   Prediction: $prediction | Truth: $truth $correct")
end